In [102]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [103]:
df=pd.read_csv("/kaggle/input/womens-ecommerce-clothing-reviews/Womens Clothing E-Commerce Reviews.csv")
df=df.fillna("nil")

In [104]:
df.sample()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
11271,11271,1096,45,Maxi dress,I really loved the dress in the picture. it is...,5,1,2,General Petite,Dresses,Dresses


In [105]:
df.isnull().sum()

Unnamed: 0                 0
Clothing ID                0
Age                        0
Title                      0
Review Text                0
Rating                     0
Recommended IND            0
Positive Feedback Count    0
Division Name              0
Department Name            0
Class Name                 0
dtype: int64

In [106]:
def text_preprocessing(text):
    data=text.lower()
    data_re=re.sub(r'\W+'," ",data)
    data_tokenize=word_tokenize(data_re)
    token=[word for word in data_tokenize if word not in stopwords.words("english")]
    return " ".join(token)

In [107]:
df["Review Text"]=df["Review Text"].apply(text_preprocessing)
df["Title"]=df["Title"].apply(text_preprocessing)

In [108]:
one_hot=OneHotEncoder(sparse_output=False)
one=one_hot.fit_transform(df[["Division Name","Department Name"]])
one=pd.DataFrame(one,columns=one_hot.get_feature_names_out())

In [109]:
Y=one_hot.fit_transform(df[["Class Name"]])

In [110]:
one

,Division Name_General,Division Name_General Petite,Division Name_Initmates,Division Name_nil,Department Name_Bottoms,Department Name_Dresses,Department Name_Intimate,Department Name_Jackets,Department Name_Tops,Department Name_Trend,Department Name_nil
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
23481,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
23482,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
23483,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
23484,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [111]:
df["Class Name"].nunique()

21

In [112]:
df.isnull().sum()

Unnamed: 0                 0
Clothing ID                0
Age                        0
Title                      0
Review Text                0
Rating                     0
Recommended IND            0
Positive Feedback Count    0
Division Name              0
Department Name            0
Class Name                 0
dtype: int64

In [113]:
tfidf=TfidfVectorizer()
t_out=tfidf.fit_transform(df["Review Text"])
t_out=t_out.toarray()
t_out=pd.DataFrame(t_out,columns=tfidf.get_feature_names_out())

In [114]:
t_out.isnull().sum().sum()

0

In [115]:
tfidf2=TfidfVectorizer()
t_out2=tfidf.fit_transform(df["Title"])
t_out2=t_out2.toarray()
t_out2=pd.DataFrame(t_out,columns=tfidf.get_feature_names_out())


In [116]:
t_out2.isnull().sum()

00              0
00p             0
10              0
100             0
12              0
            ...  
zippered        0
zippers         0
zippery     23486
zoe         23486
zowie       23486
Length: 3634, dtype: int64

In [117]:
null_columns=[]
na_columns = t_out2.columns[t_out2.isnull().all()]
if len(na_columns) > 0:
    null_columns.extend(na_columns)
t_out3 = t_out2.drop(columns=null_columns)


In [118]:
new_df1=pd.concat([df,one,t_out,t_out3],axis=1)

In [119]:
t_out3.isnull().sum().sum()

0

In [120]:
t_out3

,00,00p,10,100,12,14,150,16,16s,1950,...,younger,youthful,yr,yuck,yummy,yup,zip,zipper,zippered,zippers
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.144868,0.127713,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
23482,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
23483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
23484,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0


In [121]:
t_out2.isnull().all()

00          False
00p         False
10          False
100         False
12          False
            ...  
zippered    False
zippers     False
zippery      True
zoe          True
zowie        True
Length: 3634, dtype: bool

In [123]:
new_df=new_df.drop(columns=["Unnamed: 0","Title","Review Text","Division Name","Department Name","Class Name"])

In [124]:
new_df

,Clothing ID,Age,Rating,Recommended IND,Positive Feedback Count,Division Name_General,Division Name_General Petite,Division Name_Initmates,Division Name_nil,Department Name_Bottoms,...,younger,youthful,yr,yuck,yummy,yup,zip,zipper,zippered,zippers
0,767,33,4,1,0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
1,1080,34,5,1,4,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
2,1077,60,3,0,0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.144868,0.127713,0.0,0.0
3,1049,50,5,1,0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
4,847,47,5,1,6,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23481,1104,34,5,1,0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
23482,862,48,3,1,0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
23483,1104,31,3,0,1,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
23484,1084,28,3,1,2,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0


##### 

In [136]:
new_df.shape

(23486, 17141)

# # Deep learning

In [146]:
from keras.layers import Dense,Input
from keras.models import Model
from keras.callbacks import EarlyStopping

In [148]:
early_stopping=EarlyStopping(
monitor='val_loss',
patience=5,
verbose=1)

In [141]:
inputs=Input(shape=(17141,))
D=Dense(32,activation="relu")(inputs)
outputs=Dense(21,activation="softmax")(D)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss="categorical_crossentropy",metrics=["accuracy"])

In [152]:
model.fit(new_df,Y,epochs=100,validation_split=0.2,callbacks=[early_stopping])

Epoch 1/100
588/588 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 1.0000 - loss: 9.1486e-05 - val_accuracy: 0.9998 - val_loss: 4.9507e-04
Epoch 2/100
588/588 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 1.0000 - loss: 6.6000e-05 - val_accuracy: 1.0000 - val_loss: 3.9364e-04
Epoch 3/100
588/588 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 1.0000 - loss: 5.5793e-05 - val_accuracy: 1.0000 - val_loss: 3.4742e-04
Epoch 4/100
588/588 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 1.0000 - loss: 4.8647e-05 - val_accuracy: 1.0000 - val_loss: 2.7168e-04
Epoch 5/100
588/588 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 1.0000 - loss: 4.6837e-05 - val_accuracy: 1.0000 - val_loss: 2.4216e-04
Epoch 6/100
588/588 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 1.0000 - loss: 3.6687e-05 - val_accuracy: 1.0000 - val_loss: 2.3877e-04
Epoch 7/100
588/588 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 1.0000 - loss: 3.1396e-05 - val_accuracy: 1.0000 - val_loss: 1.8807e-04
Epoch 8/100
588/588 ━━━━━━━━━━━━━━━━━━━━ 

In [153]:
loss,accuracy=model.evaluate(new_df.iloc[100:1000],Y[100:1000])

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 2.4563e-06
